In [2]:
!wget https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/item.json

--2023-10-10 18:17:45--  https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/item.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 644735 (630K) [text/plain]
Saving to: ‘item.json’

item.json           100%[===================>] 629.62K  --.-KB/s    in 0.06s   

2023-10-10 18:17:45 (9.96 MB/s) - ‘item.json’ saved [644735/644735]



In [52]:
import json
import requests
import time
import pandas as pd

In [42]:
f = open('./item.json')
data = json.load(f)
f.close()

In [43]:
data.keys()

dict_keys(['key', 'total', 'no_more', 'items'])

In [ ]:
data['items'][0]

In [10]:
list_items = []

for item in data['items']:
  item_info = {}

  item_info['itemid'] = item['itemid']
  item_info['shopid'] = item['shopid']

  list_items.append(item_info)

In [11]:
list_items[0]

{'itemid': 22054458454, 'shopid': 76448301}

In [48]:
url_to_get_review = 'https://shopee.vn/api/v2/item/get_ratings?exclude_filter=1&filter=0&filter_size=0&flag=1&fold_filter=0&itemid={}&offset=0&relevant_reviews=false&request_source=2&shopid={}&tag_filter=&type={}&variation_filters='

review_rate_data = []

for label in [0, 1]:
  for item in list_items:
    res = requests.get(url_to_get_review.format(item['itemid'],
                                                item['shopid'],
                                                label))
    time.sleep(1)
    list_reviews = res.json()['data']['ratings']

    if list_reviews:
      for review in list_reviews:
        data_ = []

        data_.append(item['itemid'])
        data_.append(item['shopid'])
        data_.append(review['comment'])
        data_.append(review['rating_star'])
        data_.append(label)

        review_rate_data.append(data_)


In [49]:
len(review_rate_data)

1646

In [51]:
review_rate_data[2]

[22054458454,
 76448301,
 'Giao khá nhanh luôn í!! Mình đặt hôm thứ 6 nay thứ 3 nhận đc mặc dù mình ở tphcm còn shop ở hà nội@@ đóng gói khá cẩn thận nguyên hộp bự luôn í bỏ cả mút bên trong rất là cẩn thân… hàng nguyên seal hi vọng là ổn !!! Vì thấy mall nên nghĩ sẽ chất lượng.',
 5,
 0]

In [58]:
df = pd.DataFrame(review_rate_data, columns=['item_id','shop_id','comment','rate_star','label'])

In [60]:
df['label'].value_counts()

0    1344
1     302
Name: label, dtype: int64

In [61]:
df.to_csv('./shopee_reviews.csv', index=False)